# Encoding sentences using Modern BERT

### Setup

In [1]:
#!pip install transformers==4.48.0
#!pip install numpy==1.24.1

In [2]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import torch
from functools import partial
import gc
import os
from datasets import load_dataset
from src.embeddings import ModernBertEncoding
from sklearn.metrics import f1_score
from tqdm import tqdm
from transformers import AutoTokenizer, ModernBertModel
from sklearn.metrics.pairwise import cosine_similarity
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Libraries options
tqdm.pandas()
pd.options.display.max_rows = 999
pd.set_option('max_colwidth', 1000)

/home/carina/anaconda3/envs/mbert_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Acess

In [3]:
# lendo o dado processado no notebook Datasets/data_treatment.ipynb
dataframe = pd.read_parquet('../Datasets/pre_processed_dataframes/clean_df.parquet')
len(dataframe)

5070

In [4]:
dataframe[['description', 'cleaned_description']].sample()

,description,cleaned_description
3399,"S&P has upgraded Netflix to investment grade, admiring the streamer’s financial profile as it continues to lead the market in an increasingly competitive OTT landscape. The ratings agency said Monday it sees the company remaining the dominant player and it took its senior unsecured debt rating from BB+ to BBB blue chip status with outlook […]",sp upgraded netflix investment grade admiring streamers financial profile continues lead market increasingly competitive ott landscape ratings agency said monday sees company remaining dominant player took senior unsecured debt rating bb bbb blue chip status outlook


In [5]:
dataframe[['full_description', 'cleaned_full_description']].sample()


,full_description,cleaned_full_description
228,"(Teleborsa) – ESI , società quotata su Euronext Growth Milan e attiva nel mercato delle energie rinnovabili come EPC (Engineering, Procurement and Construction) e System Integrator, ha firmato un accordo quadro con un primario gruppo del settore energetico a livello nazionale e internazionale per la realizzazione di 6 impianti fotovoltaici per un valore complessivo di 25,3 milioni di euro. L’inizio delle attività è previsto per il mese di novembre 2021 e la commessa sarà di competenza per 2,5 milioni di euro dell’esercizio 2021 e per 22,8 milioni di euro dell’esercizio 2022. Il progetto prevede l’installazione di moduli fotovoltaici ad alta efficienza con l’utilizzo di tracker monoassiali e un’architettura complessiva in configurazione inverter di stringa con cabine di raccolta interne al campo solare, spiega la società in una nota. “Siamo onorati di poter aggiungere al nostro albo clienti uno dei primari gruppi nel settore energetico – ha commentato Stefano Plocco, fondatore e amm...",teleborsa esi societ quotata euronext growth milan attiva mercato energie rinnovabili epc engineering procurement and construction system integrator firmato accordo quadro primario gruppo settore energetico livello nazionale internazionale realizzazione impianti fotovoltaici valore complessivo milioni euro linizio attivit previsto mese novembre commessa sar competenza milioni euro dellesercizio milioni euro dellesercizio progetto prevede linstallazione moduli fotovoltaici alta efficienza lutilizzo tracker monoassiali unarchitettura complessiva configurazione inverter stringa cabine raccolta interne campo solare spiega societ nota onorati poter aggiungere albo clienti primari gruppi settore energetico commentato stefano plocco fondatore amministratore delegato esi laccordo subito rallentamento causa dellaumento costo materie prime dovuto pandemia vedr linizio realizzazione prime due centrali totale mwp gi partire prossimo novembre foto american public power association on unsplash


### Encoding Data

In [6]:
bert = ModernBertEncoding(model_path="answerdotai/ModernBERT-base")

In [7]:
embeddings_1 = bert.encode('trabalhando com codificação e similaridade de texto')
embeddings_2 = bert.encode('working on coding and text similarity')

In [11]:
cosine_similarity(embeddings_1, embeddings_2)[0][0]

0.79073924

In [13]:
dataframe['m_bert_embeddings_description'] = dataframe['cleaned_description'].progress_apply(bert.encode)

100%|██████████| 5070/5070 [04:46<00:00, 17.68it/s]


In [14]:
dataframe['m_bert_embeddings_full_description'] = dataframe['cleaned_full_description'].progress_apply(bert.encode)

100%|██████████| 5070/5070 [35:58<00:00,  2.35it/s]  


#### Save Processed Dataframe

In [20]:
dataframe.to_parquet('../Datasets/pre_processed_dataframes/embeddings_df.parquet', index=False)